# Capstone Project - The Battle of the Neighborhoods
### Applied Data Science Capstone by IBM/Coursera

In [1]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

Importing libraries

In [2]:
!pip install folium
import folium

print('Folium installed and imported!')

     |████████████████████████████████| 93 kB 2.3 MB/s eta 0:00:011
Folium installed and imported!


In [3]:
df_NYincidents = pd.read_csv('https://data.cityofnewyork.us/api/views/5uac-w243/rows.csv')

print('Dataset downloaded and read into a pandas dataframe!')

Dataset downloaded and read into a pandas dataframe!


In [4]:
df_NYincidents.head()

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,...,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column
0,797931560,33,MANHATTAN,09/16/2020,00:30:00,NaN,NaN,COMPLETED,NaN,NaN,...,NaN,65+,WHITE,F,998612,244331,40.837304,-73.948099,"(40.83730401400004, -73.94809867999999)",POINT (-73.94809867999999 40.83730401400004)
1,664557183,32,MANHATTAN,06/24/2020,21:20:00,06/24/2020,21:30:00,COMPLETED,NaN,NaN,...,NaN,18-24,BLACK,F,1000587,239536,40.824140,-73.940973,"(40.82413968200007, -73.94097291399999)",POINT (-73.94097291399999 40.82413968200007)
2,453437883,47,BRONX,09/28/2020,20:00:00,NaN,NaN,COMPLETED,NaN,NaN,...,NaN,25-44,BLACK,F,1030609,260452,40.881441,-73.832354,"(40.88144124600007, -73.83235353999999)",POINT (-73.83235353999999 40.88144124600007)
3,949043828,43,BRONX,07/26/2020,01:08:00,NaN,NaN,COMPLETED,NaN,NaN,...,NaN,25-44,UNKNOWN,M,1024642,243622,40.835277,-73.854033,"(40.83527732600004, -73.85403335799998)",POINT (-73.85403335799998 40.83527732600004)
4,604509546,14,MANHATTAN,09/26/2020,19:30:00,NaN,NaN,COMPLETED,NaN,NaN,...,NaN,25-44,WHITE,M,985537,214230,40.754697,-73.995356,"(40.75469651000003, -73.99535613299997)",POINT (-73.99535613299997 40.75469651000003)


In [5]:
df_NYincidents.shape

(306656, 36)

In [6]:
df_NYincidents = df_NYincidents[['BORO_NM','Latitude','Longitude']]
df_NYincidents.head()

,BORO_NM,Latitude,Longitude
0,MANHATTAN,40.837304,-73.948099
1,MANHATTAN,40.824140,-73.940973
2,BRONX,40.881441,-73.832354
3,BRONX,40.835277,-73.854033
4,MANHATTAN,40.754697,-73.995356


In [7]:
df_NYincidents.shape

(306656, 3)

In [8]:
df_NYincidents['BORO_NM'].value_counts()

BROOKLYN         88949
MANHATTAN        71982
BRONX            66982
QUEENS           65734
STATEN ISLAND    12645
Name: BORO_NM, dtype: int64

It would have been better if there is zip catergory showing us crimes occurence in respective zip code, we could have analysed the crime rate in the neighborhood more precisly 

306656 is too much of data to be shown of the map so we will be randomly picking up 200 of them.

In [9]:
df_NYincidents = df_NYincidents.sample(n=1000).reset_index(drop=True)
df_NYincidents.shape

(1000, 3)

In [10]:
df_NYincidents = df_NYincidents.rename(columns={'BORO_NM': 'Borough'}) #Renaming column
df_NYincidents.head()

,Borough,Latitude,Longitude
0,MANHATTAN,40.713830,-74.005564
1,QUEENS,40.701295,-73.741511
2,BROOKLYN,40.701404,-73.942246
3,BROOKLYN,40.624702,-73.939071
4,MANHATTAN,40.757346,-73.980181


In [11]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [12]:
ny_address = 'New York'

ny_geolocator = Nominatim(user_agent="to_explorer")
ny_location = ny_geolocator.geocode(ny_address)
ny_latitude = ny_location.latitude
ny_longitude = ny_location.longitude
print('The geograpical coordinate of New York are {}, {}.'.format(ny_latitude, ny_longitude))

The geograpical coordinate of New York are 40.7127281, -74.0060152.


Showing New York Map

In [13]:
map_ny = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=12)
map_ny

Showing location of NYU on map

In [14]:
nyu_address = 'New York University'

nyu_geolocator = Nominatim(user_agent="to_explorer")
nyu_location = nyu_geolocator.geocode(nyu_address)
nyu_latitude = nyu_location.latitude
nyu_longitude = nyu_location.longitude
print('The geograpical coordinate of NYU are {}, {}.'.format(nyu_latitude, nyu_longitude))

The geograpical coordinate of NYU are 40.72925325, -73.99625393609625.


In [15]:
folium.CircleMarker(
    [nyu_latitude, nyu_longitude],
    radius=7,
    color='red',
    popup='NYU',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_ny)

map_ny

In [16]:
from folium import plugins

# instantiate a mark cluster object for the incidents in the dataframe
data_points = plugins.MarkerCluster().add_to(map_ny)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng in zip(df_NYincidents.Latitude, df_NYincidents.Longitude):
    folium.Marker(
        location=[lat, lng],
        icon=None,
    ).add_to(data_points)

# display map
map_ny

After clustering the places where crimes took place, it seems that places around NYU have comparatively lower level of crime rate, so there's much choice in trying to pinpoint a specific area. I'm interested in Chinatown since I am of Chinese lineage. But it seems that in terms of being safe, upper west side should be a good place. Now it's time to look for restaurants and arcades ;)

In [17]:
import requests # library to handle requests

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [18]:
CLIENT_ID = '1SUTYIGJYXP1E13T0D2WT0ZIMXOGEQ35N4KQQ5ODXL1RUECC' # your Foursquare ID
CLIENT_SECRET = 'EWKJSO0BMJC14EX3RHXZUY45SAZGP23M3SXLMGPD2ZPBQT0G' # your Foursquare Secret
ACCESS_TOKEN = 'ULMV3VA5B1FFUXJIUCPMKEGSUSJHIUSWDRH5T3UB1Y1LZDJJ' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1SUTYIGJYXP1E13T0D2WT0ZIMXOGEQ35N4KQQ5ODXL1RUECC
CLIENT_SECRET:EWKJSO0BMJC14EX3RHXZUY45SAZGP23M3SXLMGPD2ZPBQT0G


In [19]:
search_query = 'Japanese'
radius = 1000 #finding places in 1km in radius from place of origin
print(search_query + ' .... OK!')

Japanese .... OK!


First, I will look from around Chinatown.

In [20]:
address1 = 'Chinatown, New York'

geolocator1 = Nominatim(user_agent="to_explorer")
location1 = geolocator1.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of Chinatown are {}, {}.'.format(latitude1, longitude1))

The geograpical coordinate of Chinatown are 40.7164913, -73.9962504.


In [21]:
folium.CircleMarker(
    [latitude1, longitude1],
    radius=7,
    color='purple',
    popup='Chinatown',
    fill = True,
    fill_color = 'purple',
    fill_opacity = 0.6
).add_to(map_ny)

map_ny

In [22]:
address2 = 'Upper West Side, New York'

geolocator2 = Nominatim(user_agent="to_explorer")
location2 = geolocator1.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
print('The geograpical coordinate of UWS are {}, {}.'.format(latitude2, longitude2))

The geograpical coordinate of UWS are 40.7870455, -73.9754163.


In [23]:
folium.CircleMarker(
    [latitude2, longitude2],
    radius=7,
    color='purple',
    popup='Upper West Side',
    fill = True,
    fill_color = 'purple',
    fill_opacity = 0.6
).add_to(map_ny)

map_ny

In [24]:
search_query = 'Japanese'
radius = 1000 #finding places in 1km in radius from place of origin
print(search_query + ' .... OK!')

Japanese .... OK!


In [25]:
url1 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url1

'https://api.foursquare.com/v2/venues/search?client_id=1SUTYIGJYXP1E13T0D2WT0ZIMXOGEQ35N4KQQ5ODXL1RUECC&client_secret=EWKJSO0BMJC14EX3RHXZUY45SAZGP23M3SXLMGPD2ZPBQT0G&ll=40.7164913,-73.9962504&oauth_token=ULMV3VA5B1FFUXJIUCPMKEGSUSJHIUSWDRH5T3UB1Y1LZDJJ&v=20180604&query=Japanese&radius=1000&limit=30'

In [26]:
results1 = requests.get(url1).json()
results1

{'meta': {'code': 200, 'requestId': '5fe5dfdb3577fa053cdbf411'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4b7736b9f964a5206b882ee3',
    'name': 'Emperor Japanese Tapas Shabu',
    'location': {'address': '96 Bowery',
     'crossStreet': 'btw Hester & Grand',
     'lat': 40.71758147589841,
     'lng': -73.99533004707291,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71758147589841,
       'lng': -73.99533004707291}],
     'distance': 144,
     'postalCode': '10013',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['96 Bowery (btw Hester & Grand)',
      'New York, NY 10013',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d142941735',
      'name': 'Asian Restaurant',
      'pluralName': 'Asian Restaurants',
      'shortName': 'Asian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',

In [27]:
# assign relevant part of JSON to venues
venues1 = results1['response']['venues']
venues1

[{'id': '4b7736b9f964a5206b882ee3',
  'name': 'Emperor Japanese Tapas Shabu',
  'location': {'address': '96 Bowery',
   'crossStreet': 'btw Hester & Grand',
   'lat': 40.71758147589841,
   'lng': -73.99533004707291,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.71758147589841,
     'lng': -73.99533004707291}],
   'distance': 144,
   'postalCode': '10013',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['96 Bowery (btw Hester & Grand)',
    'New York, NY 10013',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d142941735',
    'name': 'Asian Restaurant',
    'pluralName': 'Asian Restaurants',
    'shortName': 'Asian',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1608900571',
  'hasPerk': False},
 {'id': '4b78ac62f964a52047dd2ee3',
  'name': 'Zutto Japanese American Pub',
  'location': {'address': '77 Hud

In [28]:
# tranform venues into a dataframe
dataframe1 = json_normalize(venues1)
dataframe1.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id,location.neighborhood
0,4b7736b9f964a5206b882ee3,Emperor Japanese Tapas Shabu,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1608900571,False,96 Bowery,btw Hester & Grand,40.717581,-73.995330,"[{'label': 'display', 'lat': 40.71758147589841...",...,United States,"[96 Bowery (btw Hester & Grand), New York, NY ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4b78ac62f964a52047dd2ee3,Zutto Japanese American Pub,"[{'id': '4bf58dd8d48988d1d1941735', 'name': 'N...",v-1608900571,False,77 Hudson St,Harrison St and Worth St.,40.718566,-74.008913,"[{'label': 'display', 'lat': 40.7185655837561,...",...,United States,"[77 Hudson St (Harrison St and Worth St.), New...",867441,https://www.seamless.com/menu/zutto-77-hudson-...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,42701794,NaN
2,5f3089ef7150dc431d2d1bd1,Pinklady Cheese Tart,"[{'id': '4bf58dd8d48988d1d0941735', 'name': 'D...",v-1608900571,False,11 Mott St,Worth St,40.713986,-73.998741,"[{'label': 'display', 'lat': 40.713986, 'lng':...",...,United States,"[11 Mott St (Worth St), New York, NY 10013, Un...",2139318,https://www.seamless.com/menu/pinklady-cheese-...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN,NaN
3,525dd5f311d24f5e606f5d4a,Japanese Anime Souvenir Toys And Games,"[{'id': '4bf58dd8d48988d128951735', 'name': 'G...",v-1608900571,False,NaN,NaN,40.715313,-73.998637,"[{'label': 'display', 'lat': 40.715313, 'lng':...",...,United States,"[New York, NY, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,52ba2914498e565780aa9115,Japanese Supermarket,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",v-1608900571,False,NaN,NaN,40.714336,-73.998701,"[{'label': 'display', 'lat': 40.714336, 'lng':...",...,United States,"[New York, NY, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe1.columns if col.startswith('location.')]
dataframe_filtered1 = dataframe1.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered1['categories'] = dataframe_filtered1.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered1.columns = [column.split('.')[-1] for column in dataframe_filtered1.columns]

dataframe_filtered1 = dataframe_filtered1[['name','categories','lat','lng','distance','formattedAddress']]

#dataframe_filtered1 = dataframe_filtered1[dataframe_filtered1["categories"].str.contains('Restaurant')]

dataframe_filtered1

,name,categories,lat,lng,distance,formattedAddress
0,Emperor Japanese Tapas Shabu,Asian Restaurant,40.717581,-73.995330,144,"[96 Bowery (btw Hester & Grand), New York, NY ..."
1,Zutto Japanese American Pub,Noodle House,40.718566,-74.008913,1093,"[77 Hudson St (Harrison St and Worth St.), New..."
2,Pinklady Cheese Tart,Dessert Shop,40.713986,-73.998741,349,"[11 Mott St (Worth St), New York, NY 10013, Un..."
3,Japanese Anime Souvenir Toys And Games,Gift Shop,40.715313,-73.998637,240,"[New York, NY, United States]"
4,Japanese Supermarket,Miscellaneous Shop,40.714336,-73.998701,316,"[New York, NY, United States]"
5,AA Japanese Ramen,Ramen Restaurant,40.715248,-73.997147,157,"[45 Bayard St, New York, NY 10013, United States]"
6,Aoi Japanese Restaurant,Japanese Restaurant,40.716120,-74.005278,762,"[325 Broadway, New York, NY 10007, United States]"
7,Ni Japanese Deli,Japanese Restaurant,40.717720,-73.988210,692,"[88 Essex St, New York, NY 10002, United States]"
8,Win 49 Japanese Restaurant,Japanese Restaurant,40.722664,-73.989067,916,"[205 Allen St, New York, NY 10002, United States]"
9,Yoshi Japanese Restaurant,Japanese Restaurant,40.722221,-73.987518,974,"[201 E Houston St (Ludlow), New York, NY 10002..."


In [30]:
dataframe_filtered1 = dataframe_filtered1.dropna()

In [31]:
dataframe_filtered1[dataframe_filtered1.categories.str.contains('Restaurant',case=False)]

,name,categories,lat,lng,distance,formattedAddress
0,Emperor Japanese Tapas Shabu,Asian Restaurant,40.717581,-73.995330,144,"[96 Bowery (btw Hester & Grand), New York, NY ..."
5,AA Japanese Ramen,Ramen Restaurant,40.715248,-73.997147,157,"[45 Bayard St, New York, NY 10013, United States]"
6,Aoi Japanese Restaurant,Japanese Restaurant,40.716120,-74.005278,762,"[325 Broadway, New York, NY 10007, United States]"
7,Ni Japanese Deli,Japanese Restaurant,40.717720,-73.988210,692,"[88 Essex St, New York, NY 10002, United States]"
8,Win 49 Japanese Restaurant,Japanese Restaurant,40.722664,-73.989067,916,"[205 Allen St, New York, NY 10002, United States]"
9,Yoshi Japanese Restaurant,Japanese Restaurant,40.722221,-73.987518,974,"[201 E Houston St (Ludlow), New York, NY 10002..."
10,Kaede Japanese Restaurant,Japanese Restaurant,40.714623,-74.007270,952,"[90 Chambers St, New York, NY 10007, United St..."
12,Okinawa Japanese Restaurant,Japanese Restaurant,40.727474,-73.999786,1258,"[496 Laguardia Pl, New York, NY 10012, United ..."
13,Tataki Japanese Restaurant,Japanese Restaurant,40.720596,-74.004930,863,"[3 Lispenard St, New York, NY 10013, United St..."
14,Masuya,Restaurant,40.721432,-73.996872,552,"[85 Kenmare St, New York, NY 10012, United Sta..."


In [32]:
# add the Japanese restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered1.lat, dataframe_filtered1.lng, dataframe_filtered1.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_ny)

# display map
map_ny


Next we find arcades 

In [33]:
search_query = 'Arcade'
radius = 1000 #finding places in 1km in radius from place of origin(NYU)
print(search_query + ' .... OK!')

Arcade .... OK!


In [34]:
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url2

'https://api.foursquare.com/v2/venues/search?client_id=1SUTYIGJYXP1E13T0D2WT0ZIMXOGEQ35N4KQQ5ODXL1RUECC&client_secret=EWKJSO0BMJC14EX3RHXZUY45SAZGP23M3SXLMGPD2ZPBQT0G&ll=40.7164913,-73.9962504&oauth_token=ULMV3VA5B1FFUXJIUCPMKEGSUSJHIUSWDRH5T3UB1Y1LZDJJ&v=20180604&query=Arcade&radius=1000&limit=30'

In [35]:
results2 = requests.get(url2).json()
venues2 = results2['response']['venues'] # assign relevant part of JSON to venues
dataframe2 = json_normalize(venues2) # tranform venues into a dataframe
dataframe2.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,5053d510e4b0719d67706d96,Chinatown Fair Video Arcade,"[{'id': '4bf58dd8d48988d1e1931735', 'name': 'A...",v-1608900572,False,8 Mott St,btwn Chatham Sq & Worth St,40.713967,-73.998716,"[{'label': 'display', 'lat': 40.71396702977465...",349,10013,US,New York,NY,United States,"[8 Mott St (btwn Chatham Sq & Worth St), New Y...",NaN
1,4fecad7d011c223bce1ad061,Two-Bit's Retro Arcade,"[{'id': '4bf58dd8d48988d1e1931735', 'name': 'A...",v-1608900572,False,153 Essex St,at Stanton St,40.720831,-73.987214,"[{'label': 'display', 'lat': 40.72083090861322...",902,10002,US,New York,NY,United States,"[153 Essex St (at Stanton St), New York, NY 10...",43388864
2,4cbf45eb28d176b0b0ef226e,Moto Arcade,"[{'id': '4bf58dd8d48988d11a941735', 'name': 'O...",v-1608900572,False,201 Mulberry St,btwn Spring & Kenmare St,40.721616,-73.996588,"[{'label': 'display', 'lat': 40.72161580999999...",571,10012,US,New York,NY,United States,"[201 Mulberry St (btwn Spring & Kenmare St), N...",NaN
3,4cc0d53922ce4688bbe33947,MotoArcade!,[],v-1608900572,False,NaN,NaN,40.721567,-73.996672,"[{'label': 'display', 'lat': 40.721567, 'lng':...",566,10012,US,New York,NY,United States,"[New York, NY 10012, United States]",NaN


In [36]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')]
dataframe_filtered2 = dataframe2.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered2['categories'] = dataframe_filtered2.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered2.columns = [column.split('.')[-1] for column in dataframe_filtered2.columns]

dataframe_filtered2 = dataframe_filtered2[['name','categories','lat','lng','distance','formattedAddress']]

#dataframe_filtered1 = dataframe_filtered1[dataframe_filtered1["categories"].str.contains('Restaurant')]

dataframe_filtered2

,name,categories,lat,lng,distance,formattedAddress
0,Chinatown Fair Video Arcade,Arcade,40.713967,-73.998716,349,"[8 Mott St (btwn Chatham Sq & Worth St), New Y..."
1,Two-Bit's Retro Arcade,Arcade,40.720831,-73.987214,902,"[153 Essex St (at Stanton St), New York, NY 10..."
2,Moto Arcade,Other Nightlife,40.721616,-73.996588,571,"[201 Mulberry St (btwn Spring & Kenmare St), N..."
3,MotoArcade!,None,40.721567,-73.996672,566,"[New York, NY 10012, United States]"


In [37]:
# add the Arcades as orange circle markers
for lat, lng, label in zip(dataframe_filtered2.lat, dataframe_filtered2.lng, dataframe_filtered2.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        popup=label,
        fill = True,
        fill_color='orange',
        fill_opacity=0.6
    ).add_to(map_ny)

# display map
map_ny

Similarly, we will be doing the same visualization in Upper West Side.

In [38]:
search_query = 'Chinese'
radius = 1000 #finding places in 1km in radius from place of origin
print(search_query + ' .... OK!')

Chinese .... OK!


In [39]:
url3 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url3

'https://api.foursquare.com/v2/venues/search?client_id=1SUTYIGJYXP1E13T0D2WT0ZIMXOGEQ35N4KQQ5ODXL1RUECC&client_secret=EWKJSO0BMJC14EX3RHXZUY45SAZGP23M3SXLMGPD2ZPBQT0G&ll=40.7870455,-73.9754163&oauth_token=ULMV3VA5B1FFUXJIUCPMKEGSUSJHIUSWDRH5T3UB1Y1LZDJJ&v=20180604&query=Chinese&radius=1000&limit=30'

In [40]:
results3 = requests.get(url3).json()
venues3 = results3['response']['venues'] # assign relevant part of JSON to venues
dataframe3 = json_normalize(venues3) # tranform venues into a dataframe
dataframe3.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.city,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,5e51490c8cec040008525a7e,Chinese Laundry,"[{'id': '4bf58dd8d48988d1fc941735', 'name': 'L...",v-1608900573,False,201 W 87th St,Amsterdam,40.788624,-73.974754,"[{'label': 'display', 'lat': 40.78862380981445...",...,New York,NY,United States,"[201 W 87th St (Amsterdam), New York, NY 10024...",NaN,NaN,NaN,NaN,NaN,NaN
1,4f43f27219836ed001915c69,Jing Hua Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1608900573,False,580 Amsterdam Ave,NaN,40.789262,-73.974128,"[{'label': 'display', 'lat': 40.789262, 'lng':...",...,New York,NY,United States,"[580 Amsterdam Ave, New York, NY 10024, United...",NaN,NaN,NaN,NaN,NaN,NaN
2,58914a1cdc332974d3c4ddee,Spice Chinese & Continental Cuisine,"[{'id': '4bf58dd8d48988d142941735', 'name': 'A...",v-1608900573,False,610 Amsterdam Ave,NaN,40.790031,-73.973511,"[{'label': 'entrance', 'lat': 40.790058, 'lng'...",...,New York,NY,United States,"[610 Amsterdam Ave, New York, NY 10024, United...",NaN,NaN,NaN,NaN,NaN,NaN
3,4bc20857abf49521f0cfc193,Chinese Gi Gong Tui-na Massage,"[{'id': '52f2ab2ebcbc57f1066b8b3c', 'name': 'M...",v-1608900573,False,Amsterdam Avenue,74th Street,40.779896,-73.980842,"[{'label': 'display', 'lat': 40.77989603981298...",...,New York,NY,United States,"[Amsterdam Avenue (74th Street), New York, NY ...",NaN,NaN,NaN,NaN,NaN,NaN
4,4f0cbfa8e4b0261c94883beb,Chinese Bodywork,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",v-1608900573,False,NaN,NaN,40.778415,-73.980722,"[{'label': 'display', 'lat': 40.77841530192093...",...,New York,NY,United States,"[New York, NY 10023, United States]",NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe3.columns if col.startswith('location.')]
dataframe_filtered3 = dataframe3.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered3['categories'] = dataframe_filtered3.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered3.columns = [column.split('.')[-1] for column in dataframe_filtered3.columns]

dataframe_filtered3 = dataframe_filtered3[['name','categories','lat','lng','distance','formattedAddress']]

dataframe_filtered3 = dataframe_filtered3.dropna()

dataframe_filtered3 = dataframe_filtered3[dataframe_filtered3["categories"].str.contains('Restaurant')].reset_index(drop = True)

dataframe_filtered3

,name,categories,lat,lng,distance,formattedAddress
0,Jing Hua Chinese Restaurant,Chinese Restaurant,40.789262,-73.974128,269,"[580 Amsterdam Ave, New York, NY 10024, United..."
1,Spice Chinese & Continental Cuisine,Asian Restaurant,40.790031,-73.973511,369,"[610 Amsterdam Ave, New York, NY 10024, United..."
2,U Like Chinese Restaurant,Chinese Restaurant,40.793198,-73.966782,999,"[747 Columbus Ave, New York, NY 10025, United ..."
3,New Kam Lai,Chinese Restaurant,40.793015,-73.971051,759,"[708 Amsterdam Ave (Btw 94th & 95th), New York..."
4,Pearls,Chinese Restaurant,40.795818,-73.969046,1114,"[796 Amsterdam Ave (W 99th St), New York, NY 1..."


In [42]:
# add the Chinese restaurants as yellow circle markers
for lat, lng, label in zip(dataframe_filtered3.lat, dataframe_filtered3.lng, dataframe_filtered3.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        popup=label,
        fill = True,
        fill_color='yellow',
        fill_opacity=0.6
    ).add_to(map_ny)

# display map
map_ny

In [43]:
search_query = 'Japanese'
radius = 1000 #finding places in 1km in radius from place of origin
url4 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results4 = requests.get(url4).json()
venues4 = results4['response']['venues'] # assign relevant part of JSON to venues
dataframe4 = json_normalize(venues4) # tranform venues into a dataframe
dataframe4.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.state,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id
0,4acea43cf964a5208ad120e3,Raku It's Japanese II,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1608900574,False,57 W 76th St,at Columbus Ave,40.779991,-73.976610,"[{'label': 'display', 'lat': 40.77999142937673...",...,NY,United States,"[57 W 76th St (at Columbus Ave), New York, NY ...",284075,https://www.seamless.com/menu/raku-ii-japanese...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,96183310
1,5dfebce716e40c000881a382,Gyu-Kaku Japanese Bbq,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1608900574,False,620 Amsterdam Ave,NaN,40.790394,-73.973230,"[{'label': 'display', 'lat': 40.790394, 'lng':...",...,NY,United States,"[620 Amsterdam Ave, New York, NY 10024, United...",2332648,https://www.seamless.com/menu/gyu-kaku-620-ams...,seamless,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_seamless_20180129.png,NaN
2,50db5d27e4b0421e79fa67c8,Japanese Art @The Met,"[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",v-1608900574,False,NaN,NaN,40.784311,-73.964938,"[{'label': 'display', 'lat': 40.78431138999523...",...,NY,United States,"[New York, NY 10024, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4e4e4aa8bd4101d0d7a6fc62,New Yoshi Japanese Cuisine,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1608900574,False,466 Amsterdam Ave,NaN,40.785496,-73.976967,"[{'label': 'display', 'lat': 40.78549575805664...",...,NY,United States,"[466 Amsterdam Ave, New York, NY 10024, United...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4e4c3bbabd413c4cc667d78e,Roppongi Japanese Restaurant,"[{'id': '4bf58dd8d48988d111941735', 'name': 'J...",v-1608900574,False,434 Amsterdam Ave,NaN,40.784475,-73.977758,"[{'label': 'display', 'lat': 40.784475, 'lng':...",...,NY,United States,"[434 Amsterdam Ave, New York, NY 10024, United...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe4.columns if col.startswith('location.')]
dataframe_filtered4 = dataframe4.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered4['categories'] = dataframe_filtered4.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered4.columns = [column.split('.')[-1] for column in dataframe_filtered4.columns]

dataframe_filtered4 = dataframe_filtered4[['name','categories','lat','lng','distance','formattedAddress']]

dataframe_filtered4 = dataframe_filtered4.dropna()

dataframe_filtered4 = dataframe_filtered4[dataframe_filtered4["categories"].str.contains('Restaurant')].reset_index(drop = True)

dataframe_filtered4

,name,categories,lat,lng,distance,formattedAddress
0,Raku It's Japanese II,Japanese Restaurant,40.779991,-73.976610,791,"[57 W 76th St (at Columbus Ave), New York, NY ..."
1,Gyu-Kaku Japanese Bbq,Japanese Restaurant,40.790394,-73.973230,415,"[620 Amsterdam Ave, New York, NY 10024, United..."
2,New Yoshi Japanese Cuisine,Japanese Restaurant,40.785496,-73.976967,216,"[466 Amsterdam Ave, New York, NY 10024, United..."
3,Roppongi Japanese Restaurant,Japanese Restaurant,40.784475,-73.977758,347,"[434 Amsterdam Ave, New York, NY 10024, United..."
4,Matsu Japanese,Japanese Restaurant,40.784531,-73.973198,336,"[483 Columbus Ave, New York, NY 10024, United ..."
5,Kouzan Japanese Restaurant,Sushi Restaurant,40.792213,-73.971664,656,"[685 Amsterdam Ave (West 93rd St), New York, N..."
6,Long Grain Thai & Japanese,Japanese Restaurant,40.793201,-73.971364,765,"[710 Amsterdam Ave, New York, NY 10025, United..."
7,Neo Sushi,Sushi Restaurant,40.786507,-73.977950,221,"[2298 Broadway (83rd St), New York, NY 10024, ..."
8,Yuki Sushi,Japanese Restaurant,40.791809,-73.972813,573,"[656 Amsterdam Ave (at W 92nd St), New York, N..."
9,Matsu,Sushi Restaurant,40.784689,-73.973497,308,"[483 Columbus Ave (btw 83rd & 84th St), New Yo..."


In [45]:
# add the Japanese restaurants as yellow circle markers
for lat, lng, label in zip(dataframe_filtered4.lat, dataframe_filtered4.lng, dataframe_filtered4.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_ny)

# display map
map_ny

In [46]:
search_query = 'Arcade'
radius = 1000 #finding places in 1km in radius from place of origin
url5 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results5 = requests.get(url5).json()
venues5 = results5['response']['venues'] # assign relevant part of JSON to venues
dataframe5 = json_normalize(venues5) # tranform venues into a dataframe
dataframe5.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.state,location.country,location.formattedAddress,location.address,location.crossStreet,location.postalCode,location.city
0,4e3dae008877b00cfc4b0ceb,Arcade,"[{'id': '4bf58dd8d48988d1e1931735', 'name': 'A...",v-1608900575,False,40.792655,-73.966712,"[{'label': 'display', 'lat': 40.79265463269098...",963,US,New York,United States,"[New York, United States]",NaN,NaN,NaN,NaN
1,4f21ba67771617369d07f9c3,Cosmic Cocktails and Space Arcade at American ...,"[{'id': '4bf58dd8d48988d192941735', 'name': 'P...",v-1608900575,False,40.781171,-73.974299,"[{'label': 'display', 'lat': 40.78117059695491...",660,US,NY,United States,"[200 Central Park W (btwn W 77th & W 81st St),...",200 Central Park W,btwn W 77th & W 81st St,10024,New York


In [47]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe5.columns if col.startswith('location.')]
dataframe_filtered5 = dataframe5.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered5['categories'] = dataframe_filtered5.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered5.columns = [column.split('.')[-1] for column in dataframe_filtered5.columns]

dataframe_filtered5 = dataframe_filtered5[['name','categories','lat','lng','distance','formattedAddress']]

dataframe_filtered5 = dataframe_filtered5.dropna()

#dataframe_filtered5 = dataframe_filtered5[dataframe_filtered4["categories"].str.contains('Restaurant')].reset_index(drop = True)

dataframe_filtered5

,name,categories,lat,lng,distance,formattedAddress
0,Arcade,Arcade,40.792655,-73.966712,963,"[New York, United States]"
1,Cosmic Cocktails and Space Arcade at American ...,Planetarium,40.781171,-73.974299,660,"[200 Central Park W (btwn W 77th & W 81st St),..."


In [48]:
# add the Arcades as orange circle markers
for lat, lng, label in zip(dataframe_filtered5.lat, dataframe_filtered5.lng, dataframe_filtered5.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='orange',
        popup=label,
        fill = True,
        fill_color='orange',
        fill_opacity=0.6
    ).add_to(map_ny)

# display map
map_ny

Chinatown seems to have more variation of arcades and restaurants and closer to campus. 
Upper West Side is a bit further away but it has less crime rate compared to area near chinatown's.